Testing the functionality of camera_feed


In [6]:
from src.input_feed.src.camera_feed import CameraFeedAsync
def jobs_manager():
    from IPython.lib.backgroundjobs import BackgroundJobManager
    from IPython.core.magic import register_line_magic
    from IPython import get_ipython

    jobs = BackgroundJobManager()

    @register_line_magic
    def job(line):
        ip = get_ipython()
        jobs.new(line, ip.user_global_ns)

    return jobs

def kill_thread(thread):
    import ctypes

    id = thread.ident
    code = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        ctypes.c_long(id),
        ctypes.py_object(SystemError)
    )
    if code == 0:
        raise ValueError('invalid thread id')
    elif code != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(
            ctypes.c_long(id),
            ctypes.c_long(0)
        )
        raise SystemError('PyThreadState_SetAsyncExc failed')

In [7]:
cam = CameraFeedAsync(0)

jobs = jobs_manager()
%job cam.start_camera_stream()

In [9]:
cam.show_frame(repeat=True)

In [5]:
cam.close_camera_device()
for thread in jobs.running:
    kill_thread(thread)
